# Working with GeoTIFFs


## Introduction
This is the first part of a series of tutorials to get users familiar with `reVX`.

`reVX` (https://github.com/NREL/reVX/) is a tool used to support the [reV model](https://github.com/NREL/reV). It provides an interface to get data ready for reV modeling, as well as extracting and formatting reV outputs. 

Most use cases of `reVX` would involve formatting geospatial raster and vector data for use within reV, as well as formatting reV outputs for uses outside reV.
This tutorial will guide you through the process of working with GeoTIFF files using the `Geotiff` handler from the `reVX` library. GeoTIFF is a popular format for geospatial data, and the `Geotiff` handler provides a simple interface to load and extract various information from these files.

We'll cover the following steps:
1. Loading a GeoTIFF file using a context manager
2. Retrieving the profile (metadata)
3. Extracting band information
4. Extracting data information
5. Accessing the data values
6. Extracting latitude and longitude coordinates


Let's get started!


Let's start with a few common imports:

In [1]:
import urllib.request
from pprint import pprint
from pathlib import Path

import plotly.graph_objects as go

## Downloading the data

Before we dive into the code, we first have to download a sample TIFF from 
[Siting Lab](https://data.openei.org/submissions/6119). 
In particular, we will be using data from {cite:t}`oedi_6121`.

If you have already downloaded the data, you can skip this step (just make sure path variables below are set correctly).
We'll start by defining the local file path destination:

In [2]:
DATA_FP = "NEXRAD_green_los.tif"

Next, we can write a short download function using `urllib`,
which is part of the Python standard library, and call it to download the data.

In [3]:
NEXRAD_TIFF_URL = "https://data.openei.org/files/6121/nexrad_4km.tif"

def download_nexrad_tiff(local_filepath):
    if Path(local_filepath).exists():
        print(f"{str(local_filepath)!r} already exists!")
        return

    urllib.request.urlretrieve(NEXRAD_TIFF_URL, local_filepath)
    print(f"Downloaded {str(local_filepath)!r}!")

download_nexrad_tiff(DATA_FP)

'NEXRAD_green_los.tif' already exists!


## Reading the data

Now let's use the `Geotiff` handler from the `reVX` library to open the TIFF file in the path defined.

### Loading a GeoTIFF file using a context manager

The following example will show the use of the handler within a context manager:

In [12]:
from reVX.handlers.geotiff import Geotiff

# We can use the Geotiff handler within a context manager
with Geotiff(DATA_FP) as geo:
    methods = [m for m in dir(geo) if not m.startswith("_")]

print("Geotiff methods:\n -", "\n - ".join(sorted(methods)))

Geotiff methods:
 - bands
 - close
 - dtype
 - iarr
 - lat_lon
 - latitude
 - longitude
 - meta
 - n_cols
 - n_rows
 - profile
 - shape
 - tiff_shape
 - values
 - write


Now we will use the `Geotiff` handler to inspect some properties of the file.

### Retrieving the profile (metadata)

We can use the `profile` attribute of the `Geotiff` class to get information on the profile:

In [14]:
with Geotiff(DATA_FP) as geo:
    profile = geo.profile
    print("GeoTIFF Profile:")
    pprint(profile)


GeoTIFF Profile:
{'blockxsize': 256,
 'blockysize': 256,
 'compress': 'lzma',
 'count': 1,
 'crs': '+init=epsg:5070',
 'driver': 'GTiff',
 'dtype': 'uint8',
 'height': 32697,
 'interleave': 'band',
 'nodata': 255.0,
 'tiled': True,
 'transform': (90.0, 0.0, -2400019.7367069316, 0.0, -90.0, 3197068.2309463923),
 'width': 52005}


### Extracting band information

We can extract the number of bands in the TIFF using the use the `bands` attribute:

In [15]:
with Geotiff(DATA_FP) as geo:
    bands = geo.bands
    print("Number of Bands:", bands)


Number of Bands: 1


### Extracting data information

We can extract information on the data in the TIFF using the following attributes:

In [16]:
with Geotiff(DATA_FP) as geo:
    # Determining the Data Type
    dtype = geo.dtype
    print("Data Type:", dtype)

    # Data shape
    shape = geo.shape
    print("Image shape:", shape)

Data Type: uint8
Image shape: (32697, 52005)


### Accessing the data values

We can extract the actual data from the TIFF using the `values` attribute:

In [17]:
with Geotiff(DATA_FP) as geo:
    # Extract data as a numpy array
    data_array = geo.values
    print(f"{type(data_array)} has a shape of {data_array.shape}")

<class 'numpy.ndarray'> has a shape of (1, 32697, 52005)


### Extracting latitude and longitude coordinates

So far we have used the `Geotiff` handler exclusively as a context manager.
However, you may also use the `Geotiff` object without a context manager. 

Here, we extract (lat, lon) coordinates of each pixel location using the `lat_lon` method of the handler:

In [18]:
# Initialize a Geotiff object
geo = Geotiff(DATA_FP)

# The lat_lon attribute returns the latitude and longitude values as a tuple
# lat, lon = geo.lat_lon

# print(f"latitude shape: {lat.shape}, longitude shape: {lon.shape}")

The latitude and longitude values can also be extracted individually by using the `.latitude` and
`.longitude`` attributes of the handler, respectively.

If using the handler without a context manager, use the `close()` method, to close the source object

In [19]:
geo.close()


## Conclusion
In this tutorial, we have walked through the basic steps to load and explore GeoTIFF files using the `Geotiff` handler from the `reVX` library within a context manager. You should now be able to:
- Retrieve metadata from a GeoTIFF file
- Extract the values as a numpy array
- Extract geographic coordinates (latitude and longitude)


